In [99]:
import numpy as np
import pandas as pd
import torch
import tensorflow

Generate dataset

- Consider the exponential function f(x)=exp(-x/c), with x=0,1,2,….,L-1 integers
- Generate many training datasets {f(0,f(1),f(2),…f(L-1)} for different values of c
- Now we solve the inverse problem: input c and see if you get the exponential.

In [88]:
#Returns a dataframe of calculated exponential functions for given C-parameters and L-angular quantum numbers
class GenerateData:
    def __init__(self, l):
        self.l = l
        self.x = np.arange(l)
        self.c = np.arange(start=0.0001, stop=1, step = 0.0001)
    
    #Evaluate function for given c-values
    @staticmethod
    def func(input, arg):
        return np.e**(-input/arg)
    
    #Create dataset: 100,000 funcs?
    def populate(self):
        func_df = pd.DataFrame(index = self.c.tolist())
        for i in self.x:
            col = []
            for j in self.c:
                col.append(GenerateData(self.l).func(input = i, arg = j))
            func_df[i] = col
        func_df['c'] = self.c
        return func_df
    
    #Set param value for integer L-value (angular momentum quantum number)
    def Data(self):
        return GenerateData(self.l).populate()

Dataframe for Exponential Function Values: Rows = C-values, Columns = Integer X-values

In [120]:
data = GenerateData(4).Data()
data

,0,1,2,3,c
0.0001,1.0,0.000000,0.000000,0.000000,0.0001
0.0002,1.0,0.000000,0.000000,0.000000,0.0002
0.0003,1.0,0.000000,0.000000,0.000000,0.0003
0.0004,1.0,0.000000,0.000000,0.000000,0.0004
0.0005,1.0,0.000000,0.000000,0.000000,0.0005
...,...,...,...,...,...
0.9995,1.0,0.367695,0.135200,0.049712,0.9995
0.9996,1.0,0.367732,0.135227,0.049727,0.9996
0.9997,1.0,0.367769,0.135254,0.049742,0.9997
0.9998,1.0,0.367806,0.135281,0.049757,0.9998


Initialize neural net

In [121]:
from torch import nn
from torch.utils.data import DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Linear stack NN with a Softmax screen - initiate with nodes equal to L-value

In [122]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(len(data.columns), 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(30, 3),
        )

    def forward(self, x):
        forward = self.linear_relu_stack(x)
        return forward

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=3, bias=True)
  )
)


Implementation of Dataloader for Tensor Values

In [123]:
#Drop indicies for labels
drop_vals = np.arange(len(data.columns)-1).tolist()

#Set training and testing datasets and dataloaders
training_data = test_data = data.to_numpy()

train_dataloader = DataLoader(training_data, shuffle = True, batch_size=1)
test_dataloader = DataLoader(test_data, shuffle = False, batch_size=1)

Model Params

In [124]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Model Training/Testing

In [149]:
for epoch in range(2):  # loop over the dataset multiple times
    count = 0
    running_loss = 0.0
    for data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        labels = data[0][len(data-1)]

        l = len(data[0])-1
        inputs = data[0][0:l]
        count+=1
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

Sample Testing of Network on Random Tensors

In [19]:
X = torch.rand(100, 1, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])
